<a href="https://colab.research.google.com/github/tumble-weed/LSTM-from-scratch/blob/master/LSTM_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch

In [0]:
seq_len,batch_size,feat_len = 5,32,100
seq = np.random.random((seq_len,batch_size,feat_len))
seq = torch.tensor(seq).float()

In [0]:
class LSTM(torch.nn.Module):
    def __init__(self,input_len,state_len,squash_fn = torch.nn.functional.tanh):
        torch.nn.Module.__init__(self)
        self.input_len = input_len
        self.state_len = state_len
        self.squash_fn = squash_fn
        
        self.layers = {}
        
        self.layers['o gate from x'] = torch.nn.Linear(input_len,state_len)
        self.layers['o gate from h'] = torch.nn.Linear(state_len,state_len,bias=False)
        
        self.layers['i gate from x'] = torch.nn.Linear(input_len,state_len)
        self.layers['i gate from h'] = torch.nn.Linear(state_len,state_len,bias=False)
        
        self.layers['r gate from x'] = torch.nn.Linear(input_len,state_len)
        self.layers['r gate from h'] = torch.nn.Linear(state_len,state_len,bias=False)
        
        self.layers['delta from x'] = torch.nn.Linear(input_len,state_len)
        self.layers['delta from h'] = torch.nn.Linear(state_len,state_len,bias=False)
        
        pass
    def forward(self,x,(h,c)):
        for x_t in x:
            i_t = self.layers['i gate from x'](x_t) + self.layers['i gate from h'](h)
            i_t = torch.nn.sigmoid(i_t)
            
            o_t = self.layers['o gate from x'](x_t) + self.layers['o gate from h'](h)
            o_t = torch.nn.sigmoid(o_t)
            
            r_t = self.layers['r gate from x'](x_t) + self.layers['r gate from h'](h)
            r_t = torch.nn.sigmoid(r_t)
            
            c_hat = self.layers['delta gate from x'](x_t) + self.layers['delta gate from h'](h)
            c_hat = self.squash_fn(c_hat)
            
            c = c*r_t + c_hat*i_t
            
            h = o_t * self.squash_fn(c)
        pass